In [4]:
from mpmath import mp, mpf, findroot, j0, besselj
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

def format_dyadic_hex(value):
    l = hex(value)[2:]
    n = 8
    x = [l[i:i + n] for i in range(0, len(l), n)]
    return "0x" + "_".join(x) + "_u128"

def print_dyadic(value):
    (s, m, e) = RealField(128)(value).sign_mantissa_exponent();
    print("DyadicFloat128 {")
    print(f"    sign: DyadicSign::{'Pos' if s >= 0 else 'Neg'},")
    print(f"    exponent: {e},")
    print(f"    mantissa: {format_dyadic_hex(m)},")
    print("},")

In [5]:
def binomial_like(n, m):
    prod = QQ(1)
    z = QQ(4)*(n**2)
    for k in range(1,m + 1):
        prod *= (z - (2*k - 1)**2)
    return prod / (QQ(2)**(2*m) * (ZZ(m).factorial()))

R = LaurentSeriesRing(RealField(300), 'x',default_prec=300)
x = R.gen()

def Pn_asymptotic(n, y, terms=10):
    # now y = 1/x
    return sum( (-1)**m * binomial_like(n, 2*m) / (QQ(2)**(2*m)) * y**(QQ(2)*m) for m in range(terms) )

def Qn_asymptotic(n, y, terms=10):
    return sum( (-1)**m * binomial_like(n, 2*m + 1) / (QQ(2)**(2*m + 1)) * y**(QQ(2)*m + 1) for m in range(terms) )

P = Pn_asymptotic(0, x, 50)
Q = Qn_asymptotic(0, x, 50)

R_series = (-Q/P)

# alpha is atan(R_series) so we're doing Taylor series atan expansion on R_series

arctan_series_Z = sum([QQ(-1)**k * x**(QQ(2)*k+1) / RealField(700)(RealField(700)(2)*k+1) for k in range(25)])
alpha_series = arctan_series_Z(R_series)

# see the series
# print(alpha_series)

for i in range(alpha_series.prec()):
    if i > 44:
        break
    coeff = alpha_series[i]
    if coeff != 0:
#         print_dyadic256(RealField(300)(f"{coeff}"))
        # if i < 16:
        # print(f"\"{RealField(300)(f"{coeff}")}\",")
        print_dyadic(RealField(300)(f"{coeff}"))
        # else:
        # print_double_double("", RealField(300)(coeff))

DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -130,
    mantissa: 0x80000000_00000000_00000000_00000000_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -131,
    mantissa: 0x85555555_55555555_55555555_55555555_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -130,
    mantissa: 0xd6999999_99999999_99999999_9999999a_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -127,
    mantissa: 0xd1ac2492_49249249_24924924_92492492_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -123,
    mantissa: 0xbbcd0fc7_1c71c71c_71c71c71_c71c71c7_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -118,
    mantissa: 0x85e8fe45_8ba2e8ba_2e8ba2e8_ba2e8ba3_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -113,
    mantissa: 0x8b5a8f33_63c4ec4e_c4ec4ec4_ec4ec4ec_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -108,
    mantissa: 0xc7661d79_9d59b555_55555555_55555555_u128,
},


In [7]:
#generate b series
def binomial_like(n, m):
    prod = QQ(1)
    z = QQ(4)*(n**2)
    for k in range(1,m + 1):
        prod *= (z - (2*k - 1)**2)
    return prod / (QQ(2)**(2*m) * (ZZ(m).factorial()))

R = LaurentSeriesRing(RealField(400), 'x',default_prec=400)
x = R.gen()

def Pn_asymptotic(n, y, terms=10):
    # now y = 1/x
    return sum( (-1)**m * binomial_like(n, 2*m) / (QQ(2)**(2*m)) * y**(QQ(2)*m) for m in range(terms) )

def Qn_asymptotic(n, y, terms=10):
    return sum( (-1)**m * binomial_like(n, 2*m + 1) / (QQ(2)**(2*m + 1)) * y**(QQ(2)*m + 1) for m in range(terms) )

P = Pn_asymptotic(0, x, 50)
Q = Qn_asymptotic(0, x, 50)

def sqrt_series(s):
    val = S.valuation()
    lc = S[val]  # Leading coefficient
    b = lc.sqrt() * x**(val // 2)

    for _ in range(5):
        b = (b + S / b) / 2
        b = b
    return b

S = (P**2 + Q**2).truncate(50)

b_series = sqrt_series(S).truncate(30)
#see the series
print(b_series)

print("beta series")
for i in range(24):
    coeff = b_series[i]
    if coeff != 0:
        # print(f"{RealField(300)(f"{coeff}")},")
        # print(double_to_hex(RealField(300)(f"{coeff}")) + ",")
        print_dyadic(RealField(300)(f"{coeff}"))
        # print_double_double("", RealField(300)(f"{coeff}"))


1.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x - 0.0625000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^2 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^3 + 0.103515625000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^4 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^5 - 0.542846679687500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^6 + 0.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000